UsingIP                    2  
LongURL                    3  
ShortURL                   2  
Symbol@                    2  
Redirecting//              2  
PrefixSuffix-              2  
SubDomains                 3  
HTTPS                      3  
DomainRegLen               2  
Favicon                    2  
NonStdPort                 2  
HTTPSDomainURL             2  
RequestURL                 2  
AnchorURL                  3  
LinksInScriptTags          3  
ServerFormHandler          3  
InfoEmail                  2  
AbnormalURL                2  
WebsiteForwarding          2  
StatusBarCust              2  
DisableRightClick          2  
UsingPopupWindow           2  
IframeRedirection          2  
AgeofDomain                2  
DNSRecording               2  
WebsiteTraffic             3  
PageRank                   2  
GoogleIndex                2  
LinksPointingToPage        3  
StatsReport                2  
class                      2  

In [ ]:
#first df
import pandas as pd

train_df = pd.read_csv(r'Dataset\url_train_df.csv')
val_df = pd.read_csv(r'Dataset\url_validation_df.csv')
test_df = pd.read_csv(r'Dataset\url_test_df.csv')



In [1]:
#second df
import pandas as pd

train_df = pd.read_csv(r'Dataset\second_url_train_df.csv')
val_df = pd.read_csv(r'Dataset\second_url_validation_df.csv')
test_df = pd.read_csv(r'Dataset\second_url_test_df.csv')


In [2]:
def summarize_dataframe(df):
    summary = pd.DataFrame({
        "DataType": df.dtypes,
        "Non-Null Count": df.count(),
        "Missing Values": df.isnull().sum(),
        "Unique Values": df.nunique(),
        "First Value": df.iloc[0],
        "Sample Values": df.apply(lambda x: x.unique()[:5])
    })
    
    # Optional: Add top 5 value counts per column as a dictionary
    value_counts = {}
    for col in df.columns:
        value_counts[col] = df[col].value_counts().head(5).to_dict()
    
    summary["Top 5 Value Counts"] = pd.Series(value_counts)
    
    return summary

summary_table = summarize_dataframe(train_df)
print(summary_table)

                     DataType  Non-Null Count  Missing Values  Unique Values  \
url                    object          455833               0         449369   
type                   object          455833               0              4   
label                    bool          455833               0              2   
url_length              int64          455833               0            509   
hostname_length         int64          455833               0            237   
path_length             int64          455833               0            384   
num_dots                int64          455833               0             34   
num_hyphens             int64          455833               0             48   
num_digits              int64          455833               0            208   
num_letters             int64          455833               0            401   
num_params              int64          455833               0             16   
num_equals              int64          4

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import numpy as np
import xgboost as xgb



In [78]:
# Install if not available
# pip install xgboost scikit-learn pandas

# Split features and labels
independent_features = ['url_length', 'hostname_length', 'path_length',
       'num_dots', 'num_hyphens', 'num_digits', 'num_letters', 'num_params',
       'num_equals', 'num_slashes', 'num_at', 'has_https', 'has_ip',
       'has_subdomain', 'has_suspicious_words', 'domain_length',
       'in_alexa_top1m', 'tld_phish_ratio', 'tld_total_frequency',
       'digit_ratio', 'special_char_ratio', 'url_entropy', 'at_in_domain',
       'double_slash_in_path']
dependet_features  = 'label'
X_train = train_df[independent_features]
y_train = train_df[dependet_features]

X_test = test_df[independent_features]
y_test = test_df[dependet_features]

In [80]:
# Create XGBoost classifier
model = xgb.XGBClassifier(
    n_estimators=100,      # number of boosting rounds
    learning_rate=0.01,     # step size shrinkage
    max_depth=30,           # tree depth
    eval_metric='logloss', # evaluation metric
    use_label_encoder=False
)

# Train the model
model.fit(X_train, y_train)
# Predict
y_pred = model.predict(X_test)

# Evaluate
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


c:\Users\rrpra\AppData\Local\Programs\Python\Python313\Lib\site-packages\xgboost\training.py:183: UserWarning:

[13:41:42] WARNING: C:\actions-runner\_work\xgboost\xgboost\src\learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.




Accuracy: 0.9694866321657274

Classification Report:
               precision    recall  f1-score   support

       False       0.97      0.99      0.98     42840
        True       0.97      0.94      0.95     22279

    accuracy                           0.97     65119
   macro avg       0.97      0.96      0.97     65119
weighted avg       0.97      0.97      0.97     65119



In [79]:
#random forest
model = RandomForestClassifier(
    n_estimators=100,   # number of trees
    max_depth=30,     # tree depth (None = auto)
    random_state=42
)

# Train model
model.fit(X_train, y_train)

# Predict on test data
y_pred = model.predict(X_test)

# Evaluate model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))


Accuracy: 0.973018627435925

Classification Report:
               precision    recall  f1-score   support

       False       0.97      0.98      0.98     42840
        True       0.97      0.95      0.96     22279

    accuracy                           0.97     65119
   macro avg       0.97      0.97      0.97     65119
weighted avg       0.97      0.97      0.97     65119



- **use Z-score normalization in coloumns so that isgood for neral networks**

In [81]:
from tqdm import tqdm
tqdm.pandas()
for df in [train_df, test_df, val_df]:
    for cols in independent_features:
        if cols == 'label':
            continue
        print(f"normalizing {cols}")
        mean = df[cols].mean()
        std = df[cols].std()
        df[cols] = df[cols].progress_apply(lambda x: (x-mean)/std)

normalizing url_length


100%|██████████| 455833/455833 [00:00<00:00, 1481858.07it/s]


normalizing hostname_length


100%|██████████| 455833/455833 [00:00<00:00, 1614382.74it/s]


normalizing path_length


...

100%|██████████| 130239/130239 [00:00<00:00, 1507399.43it/s]


normalizing at_in_domain


100%|██████████| 130239/130239 [00:00<00:00, 1544858.48it/s]


normalizing double_slash_in_path


100%|██████████| 130239/130239 [00:00<00:00, 1504853.88it/s]


In [96]:
import torch 
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader

X_train = train_df[independent_features]
y_train = train_df[dependet_features]
X_val = val_df[independent_features]
y_val = val_df[dependet_features]

X_test = test_df[independent_features]
y_test = test_df[dependet_features]



# Convert features to float tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_val_tensor   = torch.tensor(X_val.values, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test.values, dtype=torch.float32)

# Convert labels to long tensors (for classification)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).squeeze()
y_val_tensor   = torch.tensor(y_val.values, dtype=torch.long).squeeze()
y_test_tensor  = torch.tensor(y_test.values, dtype=torch.long).squeeze()


In [97]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# Move tensors to device
X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_val_tensor   = X_val_tensor.to(device)
y_val_tensor   = y_val_tensor.to(device)
X_test_tensor  = X_test_tensor.to(device)
y_test_tensor  = y_test_tensor.to(device)


Using device: cuda


In [98]:


# Make sure your labels are LongTensor and zero-indexed
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long).squeeze()
y_val_tensor   = torch.tensor(y_val.values, dtype=torch.long).squeeze()
y_test_tensor  = torch.tensor(y_test.values, dtype=torch.long).squeeze()

# Convert features to float tensors
X_train_tensor = torch.tensor(X_train.values, dtype=torch.float32)
X_val_tensor   = torch.tensor(X_val.values, dtype=torch.float32)
X_test_tensor  = torch.tensor(X_test.values, dtype=torch.float32)
batch_size = 512  # you can tune this (128–2048 typical)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
val_dataset   = TensorDataset(X_val_tensor, y_val_tensor)

train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

# Number of classes
num_classes = len(y_train_tensor.unique())

# Simple FNN for classification
class SimpleFNN(nn.Module):
    def __init__(self, input_size, hidden_layers, output_size):
        super(SimpleFNN, self).__init__()
        self.input = nn.Linear(input_size, hidden_size[0])
        self.output = nn.Linear(hidden_layers[-1], output_size)
        self.relu = nn.ReLU()
        self.hiddenlayers = nn.ModuleList()
        for i in range(len(hidden_layers)-1):
            self.hiddenlayers.append(nn.Linear(hidden_layers[i], hidden_layers[i+1]))
            

    def forward(self, x):
        x = self.input(x)
        x = self.relu(x)
        for hidden in self.hiddenlayers:
            x = hidden(x)
            x = self.relu(x)
        x = self.output(x)  # raw logits
        return x

input_size = len(independent_features)
hidden_size = [128, 64, 32,16,8]
output_size = num_classes  # must match number of classes

model = SimpleFNN(input_size, hidden_size, output_size)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Training loop
Val_loss = []
Val_accuracy = []
Train_accuracy = []
Train_loss = []
epochs = 500

# Move model to device
model = model.to(device)

# Move tensors to device
X_train_tensor = X_train_tensor.to(device)
y_train_tensor = y_train_tensor.to(device)
X_val_tensor   = X_val_tensor.to(device)
y_val_tensor   = y_val_tensor.to(device)
X_test_tensor  = X_test_tensor.to(device)
y_test_tensor  = y_test_tensor.to(device)
Val_loss = []
Val_accuracy = []
Train_accuracy = []
Train_loss = []
epochs = 500

# Move model to device
model = model.to(device)

for epoch in range(epochs):
    model.train()
    running_loss = 0.0
    correct = 0
    total = 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * X_batch.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

    epoch_loss = running_loss / total
    epoch_acc = correct / total
    Train_loss.append(epoch_loss)
    Train_accuracy.append(epoch_acc)

    # Validation phase
    model.eval()
    val_loss = 0.0
    val_correct = 0
    val_total = 0

    with torch.no_grad():
        for X_val_batch, y_val_batch in val_loader:
            X_val_batch, y_val_batch = X_val_batch.to(device), y_val_batch.to(device)
            val_outputs = model(X_val_batch)
            loss = criterion(val_outputs, y_val_batch)
            val_loss += loss.item() * X_val_batch.size(0)
            _, val_predicted = torch.max(val_outputs, 1)
            val_correct += (val_predicted == y_val_batch).sum().item()
            val_total += y_val_batch.size(0)

    val_epoch_loss = val_loss / val_total
    val_epoch_acc = val_correct / val_total
    Val_loss.append(val_epoch_loss)
    Val_accuracy.append(val_epoch_acc)

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}] | "
              f"Train Loss: {epoch_loss:.4f} | "
              f"Train Acc: {epoch_acc*100:.2f}% | "
              f"Val Loss: {val_epoch_loss:.4f} | "
              f"Val Acc: {val_epoch_acc*100:.2f}%")

# Sample predictions on test set
model.eval()
with torch.no_grad():
    test_outputs = model(X_test_tensor.to(device))
    test_preds = torch.argmax(test_outputs, dim=1)
    print("Sample predictions:", test_preds[:5])

Epoch [10/500] | Train Loss: 0.0916 | Train Acc: 96.70% | Val Loss: 0.0971 | Val Acc: 96.53%
Epoch [20/500] | Train Loss: 0.0837 | Train Acc: 96.97% | Val Loss: 0.0878 | Val Acc: 96.92%
Epoch [30/500] | Train Loss: 0.0804 | Train Acc: 97.11% | Val Loss: 0.0867 | Val Acc: 96.89%
Epoch [40/500] | Train Loss: 0.0770 | Train Acc: 97.22% | Val Loss: 0.0866 | Val Acc: 96.98%
Epoch [50/500] | Train Loss: 0.0755 | Train Acc: 97.25% | Val Loss: 0.0882 | Val Acc: 96.83%
Epoch [60/500] | Train Loss: 0.0754 | Train Acc: 97.27% | Val Loss: 0.0821 | Val Acc: 97.11%
Epoch [70/500] | Train Loss: 0.0743 | Train Acc: 97.31% | Val Loss: 0.0854 | Val Acc: 97.02%
Epoch [80/500] | Train Loss: 0.0711 | Train Acc: 97.42% | Val Loss: 0.0852 | Val Acc: 97.09%
Epoch [90/500] | Train Loss: 0.0703 | Train Acc: 97.45% | Val Loss: 0.0859 | Val Acc: 97.15%
Epoch [100/500] | Train Loss: 0.0718 | Train Acc: 97.35% | Val Loss: 0.0890 | Val Acc: 97.11%
Epoch [110/500] | Train Loss: 0.0718 | Train Acc: 97.39% | Val Loss: 

KeyboardInterrupt: 

In [99]:
import plotly.graph_objects as go
epochs_range = list(range(1, len(Train_loss) + 1))

fig_loss = go.Figure()


fig_loss.add_trace(go.Scatter(x=epochs_range, y=Train_loss, mode='lines+markers', name='Train Loss'))
fig_loss.add_trace(go.Scatter(x=epochs_range, y=Val_loss, mode='lines+markers', name='validation Loss'))
fig_loss.show()

fig_accuracy = go.Figure()


fig_accuracy.add_trace(go.Scatter(x=epochs_range, y=Train_accuracy, mode='lines+markers', name='Train Loss'))
fig_accuracy.add_trace(go.Scatter(x=epochs_range, y=Val_accuracy, mode='lines+markers', name='validation Loss'))
fig_accuracy.show()

In [100]:
# Predict on test data
y_pred = model(X_test_tensor)

# Evaluate model

y_pred = y_pred.cpu()
y_pred = [1 if o > z else 0 for z,o in y_pred]
with torch.no_grad():
    print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))

Accuracy: 0.9724811498948079

Classification Report:
               precision    recall  f1-score   support

       False       0.98      0.98      0.98     42840
        True       0.96      0.96      0.96     22279

    accuracy                           0.97     65119
   macro avg       0.97      0.97      0.97     65119
weighted avg       0.97      0.97      0.97     65119



# Dataset 1

### 🧮 Accuracy without Z-score Normalization (XGBoost)

> **Accuracy:** `0.833455602268559`

| Class | Precision | Recall | F1-Score | Support |
|:------|:-----------|:--------|:----------|:----------|
| **0** | 0.85 | 0.81 | 0.83 | 35239 |
| **1** | 0.82 | 0.85 | 0.84 | 35643 |
| **Accuracy** |  |  | **0.83** | 70882 |
| **Macro Avg** | 0.83 | 0.83 | 0.83 | 70882 |
| **Weighted Avg** | 0.83 | 0.83 | 0.83 | 70882 |




### 🧮 Accuracy without Z-score Normalization (RandomForest)

> **Accuracy:** `0.8248920741514065`

| Class | Precision | Recall | F1-Score | Support |
|:------|:----------|:------|:---------|:--------|
| **0** | 0.83      | 0.81  | 0.82     | 35239   |
| **1** | 0.82      | 0.84  | 0.83     | 35643   |
| **Accuracy** |           |        | **0.82** | 70882   |
| **Macro Avg** | 0.83      | 0.82  | 0.82     | 70882   |
| **Weighted Avg** | 0.83      | 0.82  | 0.82     | 70882   |



### 🧮 Accuracy with Z-score Normalization (FNN)

> **Accuracy:** `0.8490589994638977`

| Class | Precision | Recall | F1-Score | Support |
|:------|:-----------|:--------|:----------|:----------|
| **0** | 0.88 | 0.81 | 0.84 | 35239 |
| **1** | 0.82 | 0.89 | 0.86 | 35643 |
| **Accuracy** |  |  | **0.85** | 70882 |
| **Macro Avg** | 0.85 | 0.85 | 0.85 | 70882 |
| **Weighted Avg** | 0.85 | 0.85 | 0.85 | 70882 |






# Dataset 2

### 🧮 Accuracy without Z-score Normalization (XGBoost)

> **Accuracy:** `0.9694866321657274`

| Class | Precision | Recall | F1-Score | Support |
|:------|:-----------|:--------|:----------|:----------|
| **0** | 0.97 | 0.99 | 0.98 | 35239 |
| **1** | 0.97 | 0.94 | 0.95 | 35643 |
| **Accuracy** |  |  | **0.97** | 70882 |
| **Macro Avg** | 0.97 | 0.96 | 0.97 | 70882 |
| **Weighted Avg** | 0.97 | 0.97 | 0.97 | 70882 |


### 🧮 Accuracy without Z-score Normalization (RandomForest)

> **Accuracy:** `0.973018627435925`

| Class | Precision | Recall | F1-Score | Support |
|:------|:----------|:------|:---------|:--------|
| **0** | 0.97      | 0.98  | 0.98     | 35239   |
| **1** | 0.97      | 0.95  | 0.96     | 35643   |
| **Accuracy** |           |        | **0.97** | 70882   |
| **Macro Avg** | 0.97      | 0.97  | 0.97     | 70882   |
| **Weighted Avg** | 0.97      | 0.97  | 0.97     | 70882   |



### 🧮 Accuracy with Z-score Normalization (FNN)

> **Accuracy:** `0.9724811498948079`

| Class | Precision | Recall | F1-Score | Support |
|:------|:-----------|:--------|:----------|:----------|
| **0** | 0.98 | 0.98 | 0.98 | 35239 |
| **1** | 0.96 | 0.96 | 0.96 | 35643 |
| **Accuracy** |  |  | **0.97** | 70882 |
| **Macro Avg** | 0.97 | 0.97 | 0.97 | 70882 |
| **Weighted Avg** | 0.97 | 0.97 | 0.97 | 70882 |



